<div style="text-align: center;">
  <img src="https://github.com/Hack-io-Data/Imagenes/blob/main/01-LogosHackio/logo_naranja@4x.png?raw=true" alt="esquema" />
</div>



# Laboratorio de Codificación de Variables Categóricas

AutoRenew, la empresa líder en la venta de coches de segunda mano, continúa con el desarrollo del modelo predictivo que ayudará a estimar el precio de los vehículos usados. Hasta ahora, has realizado un análisis exploratorio, gestionado los valores nulos y tratado los outliers en el conjunto de datos. El siguiente paso en el proceso de preprocesamiento es manejar las variables categóricas para que el modelo de machine learning pueda interpretarlas correctamente.

En este laboratorio, te centrarás en la **codificación de variables categóricas**. Dado que los modelos de machine learning requieren datos numéricos para funcionar, es crucial convertir las variables categóricas en un formato que los algoritmos puedan procesar. Trabajarás con el mismo conjunto de datos que has utilizado en los laboratorios anteriores, enfocándote ahora en transformar las columnas categóricas de manera efectiva.

**Instrucciones:**

1. **Identificación de Variables Categóricas:** Revisa el conjunto de datos e identifica las columnas que contienen variables categóricas. Estas son las columnas que contienen texto o categorías que necesitan ser transformadas en valores numéricos para ser utilizadas en el modelo de machine learning.

2. **Selección de Técnicas de Codificación:** Decide qué técnica(s) de codificación aplicar a cada variable categórica. Algunas de las opciones que tienes son:

   - **Label Encoding:** Asigna un número único a cada categoría dentro de una variable.

   - **One-Hot Encoding:** Crea nuevas columnas binarias (0/1) para cada categoría única en una variable.

   - **Target Encoding:** Utiliza la media del target (en este caso, el `price`) para codificar las categorías, si consideras que podría ser útil.

3. **Aplicación de la Codificación:** Implementa las técnicas seleccionadas en las variables categóricas. Asegúrate de revisar cómo estas transformaciones afectan el conjunto de datos y de mantener un registro de los cambios realizados.

4. **Verificación del Conjunto de Datos Transformado:** Una vez que hayas aplicado la codificación, verifica que el conjunto de datos resultante esté listo para ser utilizado en el modelo de machine learning. Asegúrate de que no queden variables categóricas sin codificar y que la estructura del conjunto de datos sea adecuada para el análisis.

**Nota:** Este laboratorio es parte de una serie continua en la que utilizas el mismo conjunto de datos a lo largo de la semana. Documenta bien los pasos que sigas, ya que esta codificación será clave para los modelos que desarrollarás más adelante.



In [1]:
import pandas as pd
import sys
sys.path.append("../..")
from src import support_encoding as se

from sklearn.preprocessing import OneHotEncoder

from category_encoders import TargetEncoder

import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv(r"../../datos/outliers_cleansed_it1.csv", index_col=0)

In [3]:
df.select_dtypes("O").columns

Index(['dateCrawled', 'name', 'seller', 'offerType', 'abtest', 'vehicleType',
       'yearOfRegistration', 'gearbox', 'model', 'monthOfRegistration',
       'fuelType', 'brand', 'notRepairedDamage', 'dateCreated', 'lastSeen'],
      dtype='object')

Omitiremos el nombre, el modelo y el de codigo postal, ya que presentan el mismo problema: muchas categorías (para futuro podríamos agrupar los codigos postales usando geopy). Al final tenemos categorías que pueden ser independientes para cada coche, lo cual no nos ayuda en la predicción de los precios en nuestro modelo.

In [4]:
df.drop(columns=['name', 'model', 'postalCode'], inplace=True)

In [5]:
df["dateCreated"] = pd.to_datetime(df["dateCreated"]).apply(lambda x: x.strftime("%Y-%m"))
df["dateCrawled"] = pd.to_datetime(df["dateCrawled"]).apply(lambda x: x.strftime("%Y-%m"))
df["lastSeen"] = pd.to_datetime(df["lastSeen"]).apply(lambda x: x.strftime("%Y-%m"))

In [6]:
cat_cols = df.select_dtypes("O").columns

In [7]:
print(cat_cols)

Index(['dateCrawled', 'seller', 'offerType', 'abtest', 'vehicleType',
       'yearOfRegistration', 'gearbox', 'monthOfRegistration', 'fuelType',
       'brand', 'notRepairedDamage', 'dateCreated', 'lastSeen'],
      dtype='object')


In [8]:
# se.plot_cats(data = df, columns=cat_cols, rv = "price" , plot_size=(16,22))

In [9]:
# asunciones = se.Asunciones(dataframe=df, columna_numerica="price")

# for categoria in cat_cols:   

#     print(f"Estamos analizando la variable {categoria.upper()}")
    
#     asunciones.identificar_normalidad(metodo = "kolmogorov") 

#     # comprobamos la homogeneidad de varianzas
#     asunciones.identificar_homogeneidad(columna_categorica = categoria)

#     # instanciamos la clase para evaluar si hay diferencias entre los distintos grupos de las variables categóricas
#     test = se.TestEstadisticos(df, "price", categoria)
#     test.run_all_tests()
#     print("\n###########################\n")

Las columnas SELLER, OFFERTYPE, ABTEST no presentan diferencias significativas entre cada una de sus categorías.

In [10]:
display(df["seller"].value_counts(normalize=True))
display(df["offerType"].value_counts(normalize=True))
display(df["abtest"].value_counts(normalize=True))

seller
private       0.999994
commercial    0.000006
Name: proportion, dtype: float64

offerType
offer      0.999986
request    0.000014
Name: proportion, dtype: float64

abtest
test       0.518322
control    0.481678
Name: proportion, dtype: float64

Vemos que en SELLER y OFFERTYPE hay una distribución clara en las categorías: vendedores privados y ofertas son lo mayoritariamente predominante. De esta forma, podemos optar por omitir estas dos columnas categóricas, ya que no podremos predecir bien los precios de aquellas excepciones que son de vendedor comercial o tipo de oferta petición.

Para la columna abtest simplemente haremos un one-hot encoder.

In [11]:
onehot = OneHotEncoder()
trans_one_hot = onehot.fit_transform(df[["abtest"]])
oh_df = pd.DataFrame(trans_one_hot.toarray(), columns=onehot.get_feature_names_out())

In [12]:
df = pd.concat([df.reset_index(drop=True), oh_df.reset_index(drop=True)], axis=1)

In [13]:
df.drop(columns=["seller", "offerType", "abtest"], inplace=True)

df.head()

,dateCrawled,price,vehicleType,yearOfRegistration,gearbox,powerCV,kilometer,monthOfRegistration,fuelType,brand,notRepairedDamage,dateCreated,lastSeen,abtest_control,abtest_test
0,2016-03,480.0,unknown,Coche Moderno,manually,-0.382606,0.0,unknown,petrol,volkswagen,unknown,2016-03,2016-04,0.0,1.0
1,2016-03,18300.0,coupe,Coche Contemporáneo,manually,1.218855,-0.5,may,diesel,audi,yes,2016-03,2016-04,0.0,1.0
2,2016-03,9800.0,suv,Coche Moderno,automatic,0.790609,-0.5,august,diesel,jeep,unknown,2016-03,2016-04,0.0,1.0
3,2016-03,1500.0,small car,Coche Moderno,manually,-0.642369,0.0,june,petrol,volkswagen,no,2016-03,2016-03,0.0,1.0
4,2016-03,3600.0,small car,Coche Moderno,manually,-0.741195,-1.2,july,diesel,skoda,no,2016-03,2016-04,0.0,1.0


In [14]:
df.reset_index(drop=True, inplace=True)

Ahora tratemos con las otras categóricas que sí presentan diferencias significativas entre las categorías. A estas las encodearemos usando el target, ya que mantiene un contexto general de los datos, usando de referencia nuestra variable respuesta.

In [15]:
df.select_dtypes("O").columns

Index(['dateCrawled', 'vehicleType', 'yearOfRegistration', 'gearbox',
       'monthOfRegistration', 'fuelType', 'brand', 'notRepairedDamage',
       'dateCreated', 'lastSeen'],
      dtype='object')

In [16]:
encoder = TargetEncoder(cols = df.select_dtypes("O").columns)
df_encoded = encoder.fit_transform(X = df, y = df["price"])

In [17]:
df_encoded.head()

,dateCrawled,price,vehicleType,yearOfRegistration,gearbox,powerCV,kilometer,monthOfRegistration,fuelType,brand,notRepairedDamage,dateCreated,lastSeen,abtest_control,abtest_test
0,6065.879563,480.0,5373.855068,4737.005685,4865.179692,-0.382606,0.0,2842.952996,5194.343734,5482.137159,3498.590743,6063.274713,7060.183256,0.0,1.0
1,6065.879563,18300.0,12837.225874,12610.525151,4865.179692,1.218855,-0.5,6210.592551,8615.459563,9051.491259,2472.504316,6063.274713,7060.183256,0.0,1.0
2,6065.879563,9800.0,13321.706814,4737.005685,10741.705529,0.790609,-0.5,6208.520949,8615.459563,11142.862821,3498.590743,6063.274713,7060.183256,0.0,1.0
3,6065.879563,1500.0,2951.394227,4737.005685,4865.179692,-0.642369,0.0,6362.532643,5194.343734,5482.137159,7187.912351,6063.274713,4845.496188,0.0,1.0
4,6065.879563,3600.0,2951.394227,4737.005685,4865.179692,-0.741195,-1.2,7032.572993,8615.459563,6544.456833,7187.912351,6063.274713,7060.183256,0.0,1.0


In [19]:
df_encoded.to_csv("../../datos/encoded_it1.csv")